In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.path import Path

from joblib import Parallel, delayed

from collections import deque

In [2]:
sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

In [3]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [4]:
from skimage.measure import find_contours

from matplotlib.path import Path

sys.path.append('/home/yuncong/Brain/preprocess/morphsnakes')
import morphsnakes

from shapely.geometry import Polygon

In [5]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'
scoremaps_rootdir = '/home/yuncong/csd395/CSHL_scoremaps_lossless/'
autoAnnotations_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_autoAnnotations_snake'
autoAnnotationViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_autoAnnotationsViz_snake'

In [6]:
stack = 'MD593'

autoAnnotations_dir = autoAnnotations_rootdir + '/' + stack
if not os.path.exists(autoAnnotations_dir):
    os.makedirs(autoAnnotations_dir)

autoAnnotationViz_dir = autoAnnotationViz_rootdir + '/' + stack
if not os.path.exists(autoAnnotationViz_dir):
    os.makedirs(autoAnnotationViz_dir)

first_bs_sec, last_bs_sec = section_range_lookup[stack]
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

dm = DataManager(stack=stack)

# labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']
# labels_to_detect = ['7n']

labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

In [7]:
section_contains_label = {}
# for sec in range(first_bs_sec, last_bs_sec+1):
#     dm = DataManager(stack=stack, section=sec)
#     try:
#         user, ts, _, res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
#         section_contains_label[sec] = set([lm['label'] for lm in res])
#     except:
#         pass

In [8]:
def find_contour_points(labelmap):
    '''
    return is (x,y)
    '''

    regions = regionprops(labelmap)

    contour_points = {}

    for r in regions:

        (min_row, min_col, max_row, max_col) = r.bbox

        padded = np.pad(r.filled_image, ((5,5),(5,5)), mode='constant', constant_values=0)

        contours = find_contours(padded, .5, fully_connected='high')
        contours = [cnt.astype(np.int) for cnt in contours if len(cnt) > 10]
        if len(contours) > 0:
#             if len(contours) > 1:
#                 sys.stderr.write('%d: region has more than one part\n' % r.label)
                
            contours = sorted(contours, key=lambda c: len(c), reverse=True)
            contours_list = [c-(5,5) for c in contours]
            contour_points[r.label] = sorted([c[np.arange(0, c.shape[0], 10)][:, ::-1] + (min_col, min_row) 
                                for c in contours_list], key=lambda c: len(c), reverse=True)
            
        elif len(contours) == 0:
#             sys.stderr.write('no contour is found\n')
            continue

    #         viz = np.zeros_like(r.filled_image)
    #         viz[pts_sampled[:,0], pts_sampled[:,1]] = 1
    #         plt.imshow(viz, cmap=plt.cm.gray);
    #         plt.show();
        
    return contour_points

In [ ]:
# for sec in range(first_detect_sec, last_detect_sec+1):
for sec in [150]:
    
    scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)
    
    print '\n'
    print sec
    
    #########################
    
    ref_sections = {}
    init_annotations = {}

    for l in labels_to_detect:

        nbrs_containing_this_label = [(abs(nbr-sec), nbr) for i, nbr in enumerate(range(sec-3,sec)+range(sec+1,sec+4)) 
                if nbr in section_contains_label and l in section_contains_label[nbr]]

        if len(nbrs_containing_this_label) == 0:
            sys.stderr.write('No neighbor within three sections has annotation of class %s.\n' % l)
            continue

        closest_nbr = sorted(nbrs_containing_this_label)[0][1]

        dm_nbr = DataManager(stack=stack, section=closest_nbr)
        user, ts, _, res = dm_nbr.load_proposal_review_result('yuncong', 'latest', 'consolidated')

        lms = [lm for lm in res if lm['label'] == l]
        if len(lms) > 1:
            sys.stderr.write('Region %s has more than one part.\n' % l)
            # use the larger one
            init_annotations[l] = sorted([(Polygon(lm['vertices']).area, lm) for lm in lms])[-1][1]
        else:
            assert len(lms) == 1
            init_annotations[l] = lms[0]
        
        ref_sections[l] = closest_nbr

    #########################

    valid_labels = set(labels_to_detect) & set(init_annotations.keys())
    print valid_labels
    
    if len(valid_labels) == 0:
        sys.stderr.write('No valid labels exist.\n')
        continue

    new_res = []
    
    for l in valid_labels:
#     for l in ['7N']:
        
        print l
        
        try:
            scoremap_roi = bp.unpack_ndarray_file(os.path.join(scoremaps_dir, 
                                                       '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                                       {'stack': stack, 'sec': sec, 'label': l}))
        except:
            sys.stderr.write('No scoremap of %s exists\n' % (l))
            continue
        
        lm = init_annotations[l]
        
        init_cnt = np.array(lm['vertices']).copy()
        init_cnt = (init_cnt/8).astype(np.int)
        
#         print init_cnt
        
#         cx, cy = np.mean(init_cnt, axis=0)
#         center = np.array([cx, cy])
#         init_cnt =  (.5*(init_cnt - center) + center).astype(np.int)
        
        dataset = stack + '_' + '%04d'%sec + '_roi1'
        
        interpolation_xmin, interpolation_xmax, \
        interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_dir, 
                                                                         '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': l})).astype(np.int)
        
#         dense_score_map_lossless = np.pad(scoremap, ((interpolation_ymin, dm.image_height-interpolation_ymax-1), 
#                                       (interpolation_xmin, dm.image_width-interpolation_xmax-1)),
#                                   mode='constant', constant_values=0)


        dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)
        dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                                interpolation_xmin:interpolation_xmax+1] = scoremap_roi
        
        scoremap = dense_score_map_lossless[::8, ::8]
        
        plt.figure();
        plt.imshow(scoremap, cmap=plt.cm.hot);
        plt.show();
        
#         init_levelset = convert_cnt_to_levelset(init_cnt, levelset_shape=scoremap.shape[:2])
        
        p = Path(init_cnt)

        xmin, ymin = init_cnt.min(axis=0) - 100
        xmax, ymax = init_cnt.max(axis=0) + 100
        
        xmin = max(xmin, 0)
        ymin = max(ymin, 0)
        xmax = min(xmax, dm.image_width/8-1)
        ymax = min(ymax, dm.image_height/8-1)

        xs, ys = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1))
        pts = np.c_[xs.flat, ys.flat]

        within = p.contains_points(pts)
        within_pts = pts[np.where(within)[0]]

        init_levelset = np.zeros((ymax+1-ymin, xmax+1-xmin))
        init_levelset[within_pts[:,1]-ymin, within_pts[:,0]-xmin] = 1.
        
        msnake = morphsnakes.MorphACWE((scoremap[ymin:ymax+1, xmin:xmax+1] > 0.001).astype(np.float), 
                                       smoothing=3, lambda1=1, lambda2=1)
        
        msnake.levelset = init_levelset.copy()

        diffs = []
        for i in range(100): 
            if i > 1:
                diff = np.count_nonzero(msnake.levelset - prev_levelset != 0)
                if i > 10:
                    diffs.append(diff)
                    if np.abs(np.mean(diffs[-5:]) - np.mean(diffs[-10:-5])) < 2:
                        break
            prev_levelset = msnake.levelset.copy()

            msnake.step()
            
#             plt.figure();
#             plt.imshow(msnake.levelset);
#             plt.show();
            
        print i

        try:
            new_cnts = find_contours(msnake.levelset, 0.5)
            if len(new_cnts) > 1:
                sys.stderr.write('More than one contour are detected from snake levelset: ' + \
                                ('%d '*len(new_cnts)+'\n') % tuple([c.shape[0] for c in new_cnts]))
#                 new_cnt = sorted([(len(c),c[:,::-1].astype(np.int)) for c in new_cnts])[-1][1]
                new_cnt = np.vstack([c[:, ::-1] for c in new_cnts])
            else:
                new_cnt = new_cnts[0][:, ::-1].astype(np.int)
        except:
            sys.stderr.write('No contour detected from snake levelset.\n')
            continue
        
        new_cnt = new_cnt + (xmin, ymin)
        new_cnt_subsampled = new_cnt[::20].copy()
        
#         n_sample_points = Polygon(new_cnt).exterior.length / 20
#         new_cnt = new_cnt[np.linspace(0, new_cnt.shape[0]-1, n_sample_points).astype(np.int)]
                
        new_lm = lm.copy()
        new_lm['vertices'] = new_cnt_subsampled.astype(np.int) * 8
        new_lm['refVertices'] = np.array(lm['vertices']).copy()
                            
        new_res.append(new_lm)
        
    ######################################
    
    timestamp = datetime.datetime.now().strftime("%m%d%Y%H%M%S")

    autoAnnotation_filepath = autoAnnotations_dir + '/%(stack)s_%(sec)04d_autoAnnotate_%(timestamp)s_consolidated.pkl' % \
                        {'stack': stack, 'sec': sec, 'timestamp': timestamp}

#     pickle.dump(new_res, open(autoAnnotation_filepath, 'w'))   
        
    ######################################
    
    
    dm = DataManager(stack=stack, section=sec)
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8]

    viz = cropped_img.copy()

    for lm in new_res:

    # for pt in init_cnt:
    #     cv2.circle(viz, tuple(pt), 2, (0,255,0,255), -1)

        ref_cnt = np.array(lm['refVertices']).astype(np.int)/8
        for xy in ref_cnt:
            cv2.circle(viz, tuple(xy), 3, (0,255,0), -1)

        new_cnt = np.array(lm['vertices'])/8
                
        for xy in new_cnt:
            cv2.circle(viz, tuple(xy), 5, (255,0,0), -1)
        cv2.polylines(viz, [new_cnt.astype(np.int)], True, (255,0,0), 2)


        if sec in section_contains_label and lm['label'] in section_contains_label[sec]:
            dm = DataManager(stack=stack, section=sec)
            user, ts, _, res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
            manual_cnt = [x['vertices'] for x in res if x['label'] == lm['label']][0]
            manual_cnt = np.array(manual_cnt)/8

            for xy in manual_cnt.astype(np.int):
                cv2.circle(viz, tuple(xy), 3, (0,0,255), -1)
            cv2.polylines(viz, [manual_cnt.astype(np.int)], True, (0,0,255), 2)


        lx, ly = np.array(lm['labelPos'])/8
        cv2.putText(viz, lm['label'], (int(lx)-10, int(ly)+10), cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 3)

#     cv2.imwrite(autoAnnotationViz_dir + '/%(stack)s_%(sec)04d_autoAnnotationViz.jpg' % \
#                 {'stack': stack, 'sec': sec}, viz[..., ::-1])

In [ ]:
plt.figure(figsize=(20,20));
plt.imshow(viz);

In [ ]:
test_volume_atlas_projected = bp.unpack_ndarray_file(volume_dir + '/%(stack)s_volume_atlasProjected.bp'%{'stack':stack})
print test_volume_atlas_projected.shape

(volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
    np.loadtxt(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), dtype=np.int)
    
# labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']
labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [ ]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled
    
# build annotation volume
section_bs_begin, section_bs_end = section_range_lookup[stack]
print section_bs_begin, section_bs_end


map_z_to_section = {}
for s in range(section_bs_begin, section_bs_end+1):
    for z in range(int(z_xy_ratio_downsampled*s) - volume_zmin, 
                   int(z_xy_ratio_downsampled*(s+1)) - volume_zmin + 1):
        map_z_to_section[z] = s

In [ ]:
# plt.imshow(test_volume_atlas_projected[..., 40])

In [ ]:
for sec in range(first_detect_sec, last_detect_sec+1):
# for sec in [139]:
    
    scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)
    
    print '\n'
    print sec
    
    #########################
        
    z_lowerlim = int(z_xy_ratio_downsampled*sec) - volume_zmin
    z_upperlim = int(z_xy_ratio_downsampled*(sec+1)) - volume_zmin -1 
    projected_annotation_labelmap = test_volume_atlas_projected[..., z_lowerlim]
    
    init_cnts = find_contour_points(projected_annotation_labelmap) # downsampled 16
    
    init_cnts = dict([(labels[label_ind], (cnts[0]+(volume_xmin, volume_ymin))*2) 
                      for label_ind, cnts in init_cnts.iteritems()])
    

    valid_labels = set(labels_to_detect) & set(init_cnts.keys())
    print valid_labels
    
    if len(valid_labels) == 0:
        sys.stderr.write('No valid labels exist.\n')
        continue

    new_res = []
    
    for l in valid_labels:
#     for l in ['sp5']:
        
        print l
        
        try:
            scoremap_roi = bp.unpack_ndarray_file(os.path.join(scoremaps_dir, 
                                                       '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                                       {'stack': stack, 'sec': sec, 'label': l}))
        except:
            sys.stderr.write('No scoremap of %s exists\n' % (l))
            continue
                        
        
        dataset = stack + '_' + '%04d'%sec + '_roi1'
        
        interpolation_xmin, interpolation_xmax, \
        interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_dir, 
                                                                         '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': l})).astype(np.int)
        
        dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)
        dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                                interpolation_xmin:interpolation_xmax+1] = scoremap_roi
        
        scoremap = dense_scoremap_lossless[::8, ::8]
        scoremap_viz = img_as_ubyte(plt.cm.hot(scoremap)[..., :3])
                
#         plt.figure();
#         plt.imshow(scoremap, cmap=plt.cm.hot);
#         plt.show();
        
#         init_levelset = convert_cnt_to_levelset(init_cnt, levelset_shape=scoremap.shape[:2])
        
        init_cnt = init_cnts[l]
        
#         cx, cy = np.mean(init_cnt, axis=0)
#         center = np.array([cx, cy])
#         init_cnt =  (2.*(init_cnt - center) + center).astype(np.int)
    
        p = Path(init_cnt)

        xmin, ymin = init_cnt.min(axis=0) - 100
        xmax, ymax = init_cnt.max(axis=0) + 100
        
        xmin = max(xmin, 0)
        ymin = max(ymin, 0)
        xmax = min(xmax, dm.image_width/8-1)
        ymax = min(ymax, dm.image_height/8-1)

        xs, ys = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1))
        pts = np.c_[xs.flat, ys.flat]

        within = p.contains_points(pts)
        within_pts = pts[np.where(within)[0]]

        init_levelset = np.zeros((ymax+1-ymin, xmax+1-xmin))
        init_levelset[within_pts[:,1]-ymin, within_pts[:,0]-xmin] = 1.
        
        scoremap_roi = scoremap[ymin:ymax+1, xmin:xmax+1] 
        
        msnake = morphsnakes.MorphACWE((scoremap_roi > 0.1).astype(np.float), 
                                       smoothing=3, lambda1=1., lambda2=1.)
        
        msnake.levelset = init_levelset.copy()
        # levelset values are either 1.0 or 0.0
        
        dq = deque([None, None])
        for i in range(1000): 
            
            # at stable stage, the levelset (thus contour) will oscilate, 
            # so instead of comparing to previous levelset, must compare to the one before the previous
            oneBefore_levelset = dq.popleft()
            
            if i > 10:
#                 print np.count_nonzero(msnake.levelset - oneBefore_levelset)
                if np.count_nonzero(msnake.levelset - oneBefore_levelset) < 3:
                    break

            dq.append(msnake.levelset)
        
            msnake.step()
            
#             if i % 10 == 0:
#                 a = scoremap_viz[ymin:ymax+1, xmin:xmax+1].copy()
#                 cnts = find_contours(msnake.levelset, .5)
#                 for cnt in cnts:
#                     for c in cnt[:,::-1]:
#                         cv2.circle(a, tuple(c.astype(np.int)), 1, (0,255,0), -1)

#                 plt.figure(figsize=(10,10));
#                 plt.imshow(a);
#                 plt.show();

        # in the final levelset, inside could be 0. or 1., hard to say        
        edge_arr = np.r_[msnake.levelset[:,0], msnake.levelset[:,-1], msnake.levelset[0], msnake.levelset[-1]]        
        pos_edge_num = np.count_nonzero(edge_arr)
        bool_arr = msnake.levelset.astype(np.bool)
        
        if pos_edge_num < len(edge_arr) - pos_edge_num:
            print 'mean inside score:', scoremap_roi[bool_arr].mean()
        else:
            print 'mean inside score:', scoremap_roi[~bool_arr].mean()
        print 'area:', np.count_nonzero(bool_arr)
        print 'snake iteration:', i

        try:
            new_cnts = find_contours(msnake.levelset, 0.5)
            if len(new_cnts) > 1:
                sys.stderr.write('More than one contour are detected from snake levelset: ' + \
                                ('%d '*len(new_cnts)+'\n') % tuple([c.shape[0] for c in new_cnts]))
#                 new_cnt = sorted([(len(c),c[:,::-1].astype(np.int)) for c in new_cnts])[-1][1]
                new_cnt = np.vstack([c[:, ::-1] for c in new_cnts])
            else:
                new_cnt = new_cnts[0][:, ::-1].astype(np.int)
        except:
            sys.stderr.write('No contour detected from snake levelset.\n')
            continue
        
        new_cnt = new_cnt + (xmin, ymin)
        new_cnt_subsampled = new_cnt[::20].copy()
                
#         n_sample_points = Polygon(new_cnt).exterior.length / 20
#         new_cnt = new_cnt[np.linspace(0, new_cnt.shape[0]-1, n_sample_points).astype(np.int)]
                
#         new_lm = lm.copy()
        new_lm = {}
        new_lm['label'] = l
        new_lm['labelPos'] = new_cnt_subsampled.mean(axis=0)
        new_lm['vertices'] = new_cnt_subsampled.astype(np.int) * 8
        new_lm['refVertices'] = np.array(init_cnt).copy()
                            
        new_res.append(new_lm)
        
    ######################################
    
    timestamp = datetime.datetime.now().strftime("%m%d%Y%H%M%S")

    autoAnnotation_filepath = autoAnnotations_dir + '/%(stack)s_%(sec)04d_autoAnnotate_%(timestamp)s_consolidated.pkl' % \
                        {'stack': stack, 'sec': sec, 'timestamp': timestamp}

#     pickle.dump(new_res, open(autoAnnotation_filepath, 'w'))   
        
    ######################################
    
    
    dm = DataManager(stack=stack, section=sec)
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8]

    viz = cropped_img.copy()
    
    
    for l in valid_labels:
        ref_cnt = init_cnts[l].astype(np.int)
        for xy in ref_cnt:
            cv2.circle(viz, tuple(xy), 3, (0,255,0), -1)
            
    for lm in new_res:

    # for pt in init_cnt:
    #     cv2.circle(viz, tuple(pt), 2, (0,255,0,255), -1)

#         ref_cnt = lm['refVertices'].astype(np.int)
#         for xy in ref_cnt:
#             cv2.circle(viz, tuple(xy), 3, (0,255,0), -1)

        new_cnt = np.array(lm['vertices'])/8
                
        for xy in new_cnt:
            cv2.circle(viz, tuple(xy), 5, (255,0,0), -1)
        cv2.polylines(viz, [new_cnt.astype(np.int)], True, (255,0,0), 2)


#         if sec in section_contains_label and lm['label'] in section_contains_label[sec]:
#             dm = DataManager(stack=stack, section=sec)
#             user, ts, _, res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
#             manual_cnt = [x['vertices'] for x in res if x['label'] == lm['label']][0]
#             manual_cnt = np.array(manual_cnt)/8

#             for xy in manual_cnt.astype(np.int):
#                 cv2.circle(viz, tuple(xy), 3, (0,0,255), -1)
#             cv2.polylines(viz, [manual_cnt.astype(np.int)], True, (0,0,255), 2)


        lx, ly = np.array(lm['labelPos'])
        cv2.putText(viz, lm['label'], (int(lx)-10, int(ly)+10), cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 3)

    cv2.imwrite(autoAnnotationViz_dir + '/%(stack)s_%(sec)04d_autoAnnotationViz.jpg' % \
                {'stack': stack, 'sec': sec}, viz[..., ::-1])

In [ ]:
plt.imshow(plt.cm.hot(scoremap_roi));
plt.colorbar();

In [ ]:
display(viz)

In [ ]:
def func(sec):
    
    global dm
    
    scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)
    
    print '\n'
    print sec
    
    #########################
    
    test_volume_atlas_projected = bp.unpack_ndarray_file(volume_dir + '/%(stack)s_volume_atlasProjected.bp'%{'stack':stack})
    
    z_lowerlim = int(z_xy_ratio_downsampled*sec) - volume_zmin
    z_upperlim = int(z_xy_ratio_downsampled*(sec+1)) - volume_zmin -1 
    projected_annotation_labelmap = test_volume_atlas_projected[..., z_lowerlim]
    
    init_cnts = find_contour_points(projected_annotation_labelmap) # downsampled 16
    
    init_cnts = dict([(labels[label_ind], (cnts[0]+(volume_xmin, volume_ymin))*2) 
                      for label_ind, cnts in init_cnts.iteritems()])
    

    valid_labels = set(labels_to_detect) & set(init_cnts.keys())
    print valid_labels
    
    if len(valid_labels) == 0:
        sys.stderr.write('No valid labels exist.\n')
        return
    
    
    return
    
    new_res = []
    
    for l in valid_labels:
        
#         print l
        
        try:
            scoremap_roi = bp.unpack_ndarray_file(os.path.join(scoremaps_dir, 
                                                       '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                                       {'stack': stack, 'sec': sec, 'label': l}))
        except:
            sys.stderr.write('No scoremap of %s exists\n' % (l))
            continue
                        
        
        dataset = stack + '_' + '%04d'%sec + '_roi1'
        
        interpolation_xmin, interpolation_xmax, \
        interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_dir, 
                                                                         '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': l})).astype(np.int)
        
        dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)
        dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                                interpolation_xmin:interpolation_xmax+1] = scoremap_roi
        
        scoremap = dense_scoremap_lossless[::8, ::8]
        scoremap_viz = img_as_ubyte(plt.cm.hot(scoremap)[..., :3])
                
        init_cnt = init_cnts[l]
        
        p = Path(init_cnt)

        xmin, ymin = init_cnt.min(axis=0) - 100
        xmax, ymax = init_cnt.max(axis=0) + 100
        
        xmin = max(xmin, 0)
        ymin = max(ymin, 0)
        xmax = min(xmax, dm.image_width/8-1)
        ymax = min(ymax, dm.image_height/8-1)

        xs, ys = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1))
        pts = np.c_[xs.flat, ys.flat]

        within = p.contains_points(pts)
        within_pts = pts[np.where(within)[0]]

        init_levelset = np.zeros((ymax+1-ymin, xmax+1-xmin))
        init_levelset[within_pts[:,1]-ymin, within_pts[:,0]-xmin] = 1.
        
        scoremap_roi = scoremap[ymin:ymax+1, xmin:xmax+1] 
        
        msnake = morphsnakes.MorphACWE((scoremap_roi > 0.1).astype(np.float), 
                                       smoothing=3, lambda1=1., lambda2=1.)
        
        msnake.levelset = init_levelset.copy()
        # levelset values are either 1.0 or 0.0
        
        dq = deque([None, None])
        for i in range(1000): 
            
            # at stable stage, the levelset (thus contour) will oscilate, 
            # so instead of comparing to previous levelset, must compare to the one before the previous
            oneBefore_levelset = dq.popleft()
            
            if i > 10:
                if np.count_nonzero(msnake.levelset - oneBefore_levelset) < 3:
                    break

            dq.append(msnake.levelset)
        
            msnake.step()

        # in the final levelset, inside could be 0. or 1., hard to say        
        edge_arr = np.r_[msnake.levelset[:,0], msnake.levelset[:,-1], msnake.levelset[0], msnake.levelset[-1]]        
        pos_edge_num = np.count_nonzero(edge_arr)
        bool_arr = msnake.levelset.astype(np.bool)
        
#         if pos_edge_num < len(edge_arr) - pos_edge_num:
#             print 'mean inside score:', scoremap_roi[bool_arr].mean()
#         else:
#             print 'mean inside score:', scoremap_roi[~bool_arr].mean()
#         print 'area:', np.count_nonzero(bool_arr)
#         print 'snake iteration:', i

        try:
            new_cnts = find_contours(msnake.levelset, 0.5)
            if len(new_cnts) > 1:
#                 sys.stderr.write('More than one contour are detected from snake levelset: ' + \
#                                 ('%d '*len(new_cnts)+'\n') % tuple([c.shape[0] for c in new_cnts]))
                new_cnt = np.vstack([c[:, ::-1] for c in new_cnts])
            else:
                new_cnt = new_cnts[0][:, ::-1].astype(np.int)
        except:
#             sys.stderr.write('No contour detected from snake levelset.\n')
            continue
        
        new_cnt = new_cnt + (xmin, ymin)
        new_cnt_subsampled = new_cnt[::20].copy()
                
        new_lm = {}
        new_lm['label'] = l
        new_lm['labelPos'] = new_cnt_subsampled.mean(axis=0)
        new_lm['vertices'] = new_cnt_subsampled.astype(np.int) * 8
        new_lm['refVertices'] = np.array(init_cnt).copy()
                            
        new_res.append(new_lm)
        
    ######################################
    
    timestamp = datetime.datetime.now().strftime("%m%d%Y%H%M%S")

    autoAnnotation_filepath = autoAnnotations_dir + '/%(stack)s_%(sec)04d_autoAnnotate_%(timestamp)s_consolidated.pkl' % \
                        {'stack': stack, 'sec': sec, 'timestamp': timestamp}

    pickle.dump(new_res, open(autoAnnotation_filepath, 'w'))   
        
    ######################################
    
    
    dm = DataManager(stack=stack, section=sec)
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8]

    viz = cropped_img.copy()
    
    
    for l in valid_labels:
        ref_cnt = init_cnts[l].astype(np.int)
        for xy in ref_cnt:
            cv2.circle(viz, tuple(xy), 3, (0,255,0), -1)
            
    for lm in new_res:

        new_cnt = np.array(lm['vertices'])/8
                
        for xy in new_cnt:
            cv2.circle(viz, tuple(xy), 5, (255,0,0), -1)
        cv2.polylines(viz, [new_cnt.astype(np.int)], True, (255,0,0), 2)


        lx, ly = np.array(lm['labelPos'])
        cv2.putText(viz, lm['label'], (int(lx)-10, int(ly)+10), cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 3)

    cv2.imwrite(autoAnnotationViz_dir + '/%(stack)s_%(sec)04d_autoAnnotationViz.jpg' % \
                {'stack': stack, 'sec': sec}, viz[..., ::-1])

In [ ]:
for sec in range(first_detect_sec, first_detect_sec+2):
    func(sec)

In [ ]:
import time
t = time.time()

# 360s for one stack
# _ = Parallel(n_jobs=16)(delayed(func)(sec) for sec in range(first_detect_sec, last_detect_sec+1))
_ = Parallel(n_jobs=2)(delayed(func)(sec) for sec in range(first_detect_sec, first_detect_sec+2))

print time.time() - t